# Weekly Merge
Notebook to facilitate the weekly merge with Maddie's Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Read in data
input_data = pd.read_csv("input_data.csv")
# maddie = pd.read_excel("Week-by-week elimination table.ods", engine='odf')
maddie = pd.read_excel("Scoring Workbook.ods", engine='odf')

In [3]:
# Tweak Maddie's show names
maddie = maddie.replace({"Bachelor":"The Bachelor"})

In [4]:
maddie.head()

,Show,Contestant,Week 1,Week 2,Week 3 1,Week 3 2,Week 4,Week 5
0,The Bachelor,Alayah,0,0,0,0,0,0
1,The Bachelor,Alexa,0,0,0,0,0,0
2,The Bachelor,Deandra,1,0,0,0,0,0
3,The Bachelor,Hannah Ann,1,1,1,1,1,1
4,The Bachelor,Jasmine,0,0,0,0,0,0


In [5]:
# Trim my data (I trust Maddie more)
# input_data.drop(columns=["Week 1", "Week 2", "Score"], inplace=True)
input_data = input_data[["Show", "Name", "file", "PickedBy", "Eliminated"]]

In [6]:
# Merge on show and contenstant
df = pd.merge(input_data, maddie, left_on=["Show", "Name"],
              right_on=['Show', "Contestant"], how='left')

In [7]:
df.Show.unique()

array(['Kids Baking Championship', 'Project Runway', 'Survivor',
       'The Bachelor', 'Top Chef'], dtype=object)

In [9]:
# Fill X and NA with 0
df.fillna(0., inplace=True)
df.replace({"X":0.}, inplace=True)

In [10]:
# Change datatypes for weeks
for col in df.columns:
    if "Week" in col:
        df[col] = df[col].astype(float)

In [11]:
# Get Scores
week_cols = list(filter(lambda x: "Week" in x, list(df.columns)))
df['Score'] = df[week_cols].sum(axis=1)

In [12]:
# Handle eliminated
last_week = week_cols[-1]
df['Eliminated'] = np.where(df[last_week] == 0, 1, 0)

In [13]:
# Fix for Top Chef (hasn't started)
df['Eliminated'] = np.where(df.Show == "Top Chef", 0, df.Eliminated)

In [14]:
df.to_csv("input_data.csv")

In [15]:
## Save to JSON
s = df.to_json(orient='records')
f = open("data.js", 'w+')
f.write("var data = " + s + ';')
f.close()